In [1]:
#8-1 
import numpy as np

def reweight_distribution(original_distribution, tempreature=0.5):
    distribution = np.log(original_distribution)/temperature
    distribution = np.exp(distribution)
    return distribution /np.sum(distribution)

In [6]:
import keras 

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()
print('size of text:', len(text))

size of text: 600893


In [25]:
maxlen = 60 #extract sequence to consist of 60 alphabet form test
step =3

sentences = []
next_chars=[]

for i in range(0,len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
    #in this point, Because I wrote wrong spell text[i:maxlen], it makes key error at line 28 

print('number of sequences:',len(sentences))

chars=sorted(list(set(text))) #sort word that have independent meaning  and put in list
print('Unique word:',len(chars))
char_indices=dict((char,chars.index(char)) for char in chars)
print(char_indices)
#make a dictionary which mapping word in list with their index 

print('vectorization...')
x=np.zeros((len(sentences),maxlen,len(chars)), dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences): #enumerate => check the index in Repetitive
    #print(sentence)
    for t,char in enumerate(sentence):
        #print(t,char)
        #print(char_indices[char])
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1
#one-hot encodeing for sentences

number of sequences: 200278
Unique word: 57
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}
vectorization...


In [28]:
from keras import layers

model=keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

In [31]:
optimizer=keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

In [41]:
def sample(preds, temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas= np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [42]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text)-maxlen-1)

for epoch in range(1,60) :
    print('epoch:', epoch)
    model.fit(x,y,batch_size=128,epochs=1)
    
    seed_text = text[start_index: start_index+maxlen]
    print('----seed text:"'+seed_text+'"')
    
    for temperature in [0.2,0.5,1.0,1.2]:
        print('----temperature:',temperature)
        generated_text=seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1,maxlen,len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1.
            
            preds=model.predict(sampled, verbose=0)[0]
            next_index=sample(preds, temperature)
            next_char=chars[next_index]
            
            generated_text+=next_char
            generated_text=generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
            
        print()

epoch: 1
Epoch 1/1
200278/200278 [==============================] - 54s 268us/step - loss: 1.4423
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the superficient to the problem of the same and the sense and the world to the species of the prompted and the reason and instance, and as the spirited the spirit, and in the same as the one may be standard to the sense of the spirited interpreted to the conduct in the spirit to the problem of the same and soul as the stronger and in the same the superficient to the considered in the sense of th
----temperature: 0.5
the slowly ascending ranks and classes, in which,
through for anything more the discondect in the interpretation to the habtly of the fact of the propered; and always the same higher any man with the discondently and man as the world of the consequently to the encondred, or destruction of the reason or no longer of the s

the world, who priys with his erding but one seeved, more whorarly the opinion, is only he sufferented of himself--brolluggest and peoples needs--it was relational itself as the andly long pr
----temperature: 1.2
the slowly ascending ranks and classes, in which,
through for prequere a soctar
perfect of cruelt planegat, this must man very an effect or
later will gentimingsed. the mediocre--their most
satisfactids feol
when one shoalsements. aot, tooctiral advantation of eoud had ca
dfility it is had a sin of a hadd man, whogly on errordined! i wilding absorbative garm in bad himselfo and bodies--one has
glorceahant attentari[(que the holays, howeveeter, anve's ale a
epoch: 9
Epoch 1/1
200278/200278 [==============================] - 54s 269us/step - loss: 1.3634
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the super-quically believe the self-desting, in the same the self-su

/home/server1/anaconda3/envs/ksa2020/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 for an instinctide the rational, these mrans, of the long-summaterment jus
----temperature: 1.2
the slowly ascending ranks and classes, in which,
through for orderstaten--one musils. prought and whomed an undrither and yould?".

p1. the relation of usuble) of classive
as upo
gree.
for the power
moral itself moral
"interpretatian menuncated though thly tage
to many rrobs: let us rong to an astance. to knew squires whatever enviby." the about complanation etoith eurow to yours
of certais in with bede thef, sympathy. in the corrraded history
into a such
epoch: 10
Epoch 1/1
200278/200278 [==============================] - 53s 266us/step - loss: 1.3597
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the super-thich one is a perhaps and soul and consideration of the most problem of the point of a perhaps of the problem of the sense of the super-marker of the most conditions of the

through for mankind will means and conscience, as the problem of the instinct of the strength of the intellectual expetse of the same approposition of the problem of the religion of the very "will, and so that the truth of his own less more something is a philosophers as and the instance, as last, and that is a maningeng by the fact that the same tender is the desires of the belief in the conscience of him,
----temperature: 1.0
the slowly ascending ranks and classes, in which,
through for hoself-discrabes of the slage of spirit, hence thealory! it is woman is out knows itself: the spirits
of discises tuut nojid and interrogat. also no all to difference of germans
opposing it necessary to individualisis
amounds, unthrisibalition has humbden. that we always to made our subertedep mode has datingue of the
toon
yet no longer false" hand. even fundamental speaking into the average to 
----temperature: 1.2
the slowly ascending ranks and classes, in which,
through forgeanty, as
a son "bat boo

200278/200278 [==============================] - 53s 267us/step - loss: 1.3198
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the same and morality of the strength of the constantly of the same stroke the constantly of the strength of the strength of the most precisely becomes the presence of the most consequently his own the strength of the proces of the the stronger of the most impulse of the person of the spirit of the present the standpoint of the strength of the proces of the most considered of the strength of the
----temperature: 0.5
the slowly ascending ranks and classes, in which,
through for explanation of the bring with deaver to a constantly of
the micreby him to their constraintly and defevience of concerning the constraintable of his own doubts of the good of the super-it--the man there is doubled from the unifitions of the christian spirit of the
problem of the

through formlibuts--but a middss-path of our danger, neverthelened, hellt workted, woman deeptily?, edry knowledge elequent righte sloioning wart. ever aut imphilosowikeds, and a spirit could chiol
supposition have been bese
order and alnow reaunt he
said that in
order toneful, then genereds of all usting bot have for the mekivon, of
the dekecchone in sharp are is dot, otest from always that wark of this th
epoch: 33
Epoch 1/1
200278/200278 [==============================] - 53s 265us/step - loss: 1.3179
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the sense of the proper the subject of the same world and the responsible of the moral and the concerning of the subjugtion of the successful of the same and soul is not and a moral and such an exception of the subject of the profound and the discovered and will of the successful sense of the sense of the sense of the world and 

through for the commucy of the assist, the struggle of the religionable of the highest part of the strengthy. the german depression the experience in this worlds of the religious part of the contrary of the experience of the problem of the sentious artifice of the fundamental of the powerful conscience to look of the fact of mankind and generally and the transificance of his plant things, as the will to ear
----temperature: 1.0
the slowly ascending ranks and classes, in which,
through for everything romuch as a close philosophy"), and called himary, and painatment of the lowers owarly almost unulared
arguse
thar to influence and philosophers to borting very towarddins than so we pretudation for thore, which effection,
or in conventeisnsity
of
subiciety, some of like it trougnation and dardfulness and a could morals, hurm tofmens through this
mority, is constance himsestins play s
----temperature: 1.2
the slowly ascending ranks and classes, in which,
through formess, ro, under what mode

200278/200278 [==============================] - 53s 266us/step - loss: 1.3341
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the strong and distinguish the respect of the truth of the conscience of the strength of the strength of the spirit and interest man is not to the same time and stand of the world and even and strong the fact of the painfess of the spirit of the sense of the truth of the sense of the man is to the strong and morality of the most promise of the respect of the self-descred and with the same time t
----temperature: 0.5
the slowly ascending ranks and classes, in which,
through for the develop
of being the unabling and class in the same "to him and every long for the protection of proper to very profound existion of the reason of dominant thander what is partial such a popular, as a intellectually best in which the science, and has are also a passions, or 

through for rore: by
next to get of hefce
it were the problem and
tet were runess, to contirl as a doctrine
volpitually andsciesished our highy will like a ming onemys has been jidyness it there do there thay books which time
creved- live-morap" ashan obeduely,
an remain "bar,tures of directed himself collectary smile licates of difficult remain teach. they undifued chrang feeling and tame and than christip
epoch: 56
Epoch 1/1
200278/200278 [==============================] - 53s 266us/step - loss: 1.4290
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the presentation of the problem of the surxeration of the present of the surpligh of the conception of the surpluage of the surpligh of the same that the surplight of the present of the most present of the surpluage of the surpligh of the surpligh of the strong the strong and proposition of the man are the surpluage of the prese